In [6]:
#Importing libraries
import pandas as pd
import re
import datetime

In [7]:
def clean_special_and_split_words(text):
    
    #Checking if str, if not making it str
    if not isinstance(text, str):
        text = str(text)
        
    #Replacing things below:
    text = text.replace('-', ' ').replace('+', ' ').replace("ö", "o")
    
    #Removing all but dots between letters
    text_cleaned = re.sub(r'[^a-zA-Z0-9.\s]', '', text)
    
    #Replacing dots between letters by space
    text_cleaned = re.sub(r'(?<=\w)\.(?=\w)', ' ', text_cleaned)
    
    text_cleaned = text_cleaned.upper()
    words = text_cleaned.split()
    
    return words


def clean_special_and_split_letter(text):
    
    #Checking if str, if not making it str
    if not isinstance(text, str):
        text = str(text)
        
    #Replacing ö by o
    text = text.replace("ö", "o")
    
    #Removing all special
    text_cleaned = re.sub(r'[^a-zA-Z0-9]', '', text)
    text_cleaned = text_cleaned.upper()
    
    return text_cleaned


def compare_cells(row):
    
    a = row['Manufacturer_implementation_clean']
    b = row['Manufacturer_client_clean']
    if any(word in a for word in b):
        return 1
    else:
        return 0
    

def match_percentage(row):
    
    match_percentage = 0
    
    try:
        
        a = str(row['Mnp_implementation_clean'])
        b = str(row['Mnp_client_clean'])
        
        if a == b:
            match_percentage = 100
        
        match_count = 0
        min_len = min(len(a), len(b))
        
        for i in range(min_len):
            if a[i] == b[i]:
                match_count += 1
        
        match_percentage = round((match_count / max(len(a), len(b))) * 100, 1)
        

    except :
        match_percentage = 0
        
    return match_percentage
    
    
def manufacturer_matching(df_merged):

    df_merged = df_merged[["Item_code", "Manufacturer_client", "Manufacturer_implementation"]]
    df_merged = df_merged.sort_values(by="Item_code")
    df_merged.dropna(inplace=True)
    df_merged['match_manufacturer'] = 0
    
    df_merged.loc[:, 'Manufacturer_client'] = df_merged['Manufacturer_client']
    df_merged.loc[:, 'Manufacturer_client_clean'] = df_merged['Manufacturer_client'].apply(clean_special_and_split_words)

    df_merged.loc[:, 'Manufacturer_implementation'] = df_merged['Manufacturer_implementation']
    df_merged.loc[:, 'Manufacturer_implementation_clean'] = df_merged['Manufacturer_implementation'].apply(clean_special_and_split_words)

    df_merged['match_manufacturer'] = df_merged.apply(compare_cells, axis=1)
    
    return df_merged


def mnp_matching(df_merged):
    
    df_merged = df_merged[["Item_code", "Mnp_client", "Mnp_implementation"]]
    df_merged = df_merged.sort_values(by="Item_code")
    df_merged.dropna(inplace=True)
    df_merged['match_mnp'] = 0
    
    df_merged.loc[:, 'Mnp_client'] = df_merged['Mnp_client']
    df_merged.loc[:, 'Mnp_client_clean'] = df_merged['Mnp_client'].apply(clean_special_and_split_letter)

    df_merged.loc[:, 'Mnp_implementation'] = df_merged['Mnp_implementation']
    df_merged.loc[:, 'Mnp_implementation_clean'] = df_merged['Mnp_implementation'].apply(clean_special_and_split_letter)
    
    df_merged['match_mnp'] = df_merged.apply(match_percentage, axis=1)
    
    return df_merged

def main():
        
    client_input_file = "SnapPart Inventory Load - Rolls Royce Annesley.xlsx"
    #implementation_input_file = input('Name of the implementation input file : ')
    #implementation_input_file = implementation_input_file + ".xlsx"
    implementation_input_file = "RR Annesley.xlsx"
    
    # Load client and implementation data
    df_client = pd.read_excel(client_input_file)
    df_client = df_client[["Item  Code", "Manufacturer", "Mfr. Part #"]]
    df_client.rename(columns={"Item  Code": "Item_code", "Manufacturer": "Manufacturer_client", "Mfr. Part #": "Mnp_client"}, inplace=True)
    
    df_implementation = pd.read_excel(implementation_input_file)
    df_implementation = df_implementation[["Item", "Manufacturer (Item) (Item)", "Mfr. Part # (Item) (Item)"]]
    df_implementation.rename(columns={"Item": "Item_code", "Manufacturer (Item) (Item)": "Manufacturer_implementation", "Mfr. Part # (Item) (Item)": "Mnp_implementation"}, inplace=True)
    
    # Merge dataframes on Item_code
    df_merged = pd.merge(df_client, df_implementation, on="Item_code", how="inner")
    
    # Matching manufacturers and part numbers
    df_match_manuf = manufacturer_matching(df_merged)
    df_match_mnp = mnp_matching(df_merged)
    
    # Merge results and save to an Excel file
    df_match = pd.merge(df_match_manuf, df_match_mnp, on="Item_code", how="inner")
    
    df_match.drop(["Mnp_implementation_clean", "Mnp_client_clean", "Manufacturer_client_clean", "Manufacturer_implementation_clean"], axis=1, inplace=True)   
     
    today_date = datetime.date.today()
    df_match.to_excel(f'RRI_match_{today_date}.xlsx', sheet_name='Results', index=False)
    
#Active SnapPart Inventory Balances for Export 8-15-2024 6-04-26 AM

In [8]:
if __name__ == "__main__":
    main()

c:\Users\misol\anaconda3\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
